## Project Title :  

## 0. Dependencies 

Data Folder:
Data/tweet.txt
Data/tweet_x.csv

Libraries:
sklearn
scipy

In [55]:
import matplotlib.pyplot as plt
import pandas as pd


from Definations import *
from Utilities import *
from sklearn.feature_selection.univariate_selection import SelectPercentile
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from sklearn.manifold import isomap
import numpy as np
from scipy.sparse import hstack



# TODO: Import the three supervised learning models from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## 1. Dataset Exploration

### Step 1: Benchmark Experiment Replication using ASTD. ###

1. It stands for Arabic Sentemental Tweet Dataset which is used for arabic social sentimental analysis .
2. Consist of 10000 tweets gathered from twitter.
3. Each tweet is Classified as: objective, subjective positive, subjective negative or subjective mixed.

### Step 2: Loading the data set in jupyter for exploration. ###

In [56]:
# Import libraries necessary for this project
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

# Import supplementary visualization code visuals.py
import visuals as vs

In [57]:
!pip install wordcloud

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [58]:
import nltk
import arabic_reshaper
from bidi.algorithm import get_display
import csv
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

##### Text File

In [60]:
import codecs
filename = codecs.open('data\Tweets.txt', 'r', encoding="utf-8")
outputfile = filename.readlines()
for line in outputfile[0:5]:
    print(line)

بعد استقالة رئيس #المحكمة_الدستورية ننتظر استقالة #رئيس_القضاء #السودان	OBJ

أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر، بمناسبة صدور أولى روايته	POS

البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام العريان الي واشنطن شئ مقرف	NEG

#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية أول فيلم استقصائي يتناول أسرار و كواليس تعرض لأول مرة حول حقيقة	OBJ

الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقولها ملل الله وكيلك تعطيني محاضرة عن الفسق والفجور بجنوب الشيشان #ليه كذا يانبع الحنان	NEUTRAL



##### CSV File

In [61]:
df = pd.read_csv('data\Tweets_x.csv', delimiter='\t',names= ["Tweet", "Classification"])

df.head()

,Tweet,Classification
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,OBJ
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL


In [62]:
df.describe()

,Tweet,Classification
count,10006,9986
unique,10002,4
top,يوسف الحسيني بالفيديو يفضح كذب قناة الجزيرة وي...,OBJ
freq,2,6675


AS it can be clearly seen, I lost some data in this process since i converted the txt into csv using excel. I removed them to keep dataset consistant but I will fix this issue later

#### Show Count of All Categories ####

In [63]:
df = df.dropna()

In [64]:
df.describe()

,Tweet,Classification
count,9986,9986
unique,9982,4
top,الإخوان يطلقون طفايات الحريق للإيحاء بإلقاء ال...,OBJ
freq,2,6675


In [65]:
# Split the dataset in training and test set:
docs_train, docs_test, y_train, y_test = train_test_split(
    df['Tweet'], df['Classification'], test_size=0.5)


# TASK: Build a vectorizer that splits strings into sequence of 1 to 3
# characters instead of word tokens
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['Tweet'])
print(X_train_counts.shape)

#return how many word has occured in a document
print(count_vect.vocabulary_.get(u'الحرية'))

(9986, 37515)
6281


In [66]:
# print(count_vect.vocabulary_)
keys = np.array(list(count_vect.vocabulary_.keys()))
values = np.array(list(count_vect.vocabulary_.values()))

words_tokenized = pd.DataFrame(values,columns=['frequency'],index=keys) 
words_tokenized.head()

,frequency
دراما,17941
استنتاجات,3519
الغربة,8174
إفساد,2405
شيفنا,20447


### Exploraing tweet contints

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:

stop_words = set(stopwords.words('arabic')) 
  
word_tokens = count_vect.vocabulary_.keys()
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 

print(len(word_tokens) )
print(len(filtered_sentence ) )

** Word Frequncy Visualization using wordcloud **

In [ ]:
entries = []
for w in word_tokens:
    reshaped_text = arabic_reshaper.reshape(w)
    bidi_text = get_display(reshaped_text)
    entries.append([bidi_text ,count_vect.vocabulary_.get(w) ] )

In [ ]:
#Generating wordcloud. Relative scaling value is to adjust the importance of a frequency word.
#See documentation: https://github.com/amueller/word_cloud/blob/master/wordcloud/wordcloud.py
wordcloud = WordCloud(font_path="font_aljazeera.ttf").generate_from_frequencies(dict(entries))

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
count = 0
for r in stop_words:
    print (r)
    if count > 10:
        break
    count+=1

In [ ]:
filename = open("data/tweets.txt") 
outputfile = filename.readlines()
for line in outputfile:
    words = line.split() 
    for r in words: 
        if r in stop_words: 
            print(r)

** we did not use the code below because we did not see any word in the stop word list **

filename = open("data/tweets.txt") 

outputfile = filename.readlines()

for line in outputfile:
    
    words = line.split() 
    
    for r in words: 
        
        if not r in stop_words: 
            
            appendFile = open('data/ncleaned_tweets.txt','a') 
            
            appendFile.write("\t"+r) 
            
    appendFile.write("\n")
    
    appendFile.close()

In [ ]:
s2 = df.Classification
print(s2.value_counts())
# s2.value_counts().plot(kind='hist') 
s2.value_counts().plot( kind="bar") 
plt.show()

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# y = np.random.rand(10,4)
# y[:,0]= np.arange(10)
# df = pd.DataFrame(y, columns=["X", "A", "B", "C"])

# ax = df.plot(x="X", y="A", kind="bar")
# df.plot(x="X", y="B", kind="bar", ax=ax, color="C2")
# df.plot(x="X", y="C", kind="bar", ax=ax, color="C3")

# plt.show()

## 2. Preprocessing

### Step 3: Data PreProcessing ###

In [ ]:
ara = AraTweet()

(Data,rating)=ara.read_clean_reviews()

In [ ]:
len(rating)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

target_rating = le.fit_transform(rating)
# list(le.inverse_transform([0,1,2,3]))

In [ ]:
target_rating.shape

In [ ]:
target_rating.data[10005]

In [ ]:
le.classes_

In [ ]:
target_rating

#### String Token Count(Bag of Words):

It takes each sentence (all the words) present in the data set in the review section and then splits each of the words present in the form of tokens. The occurrence of these tokens in the whole data set are counted in such a way that the count of the occurrence of each token in a positive and negative feedback (in balanced dataset) or positive, negative and neutral (in unbalanced dataset) are collected separately. Finally, the word frequency of the tokens is calculated.

code:  https://stackoverflow.com/questions/653887/equivalent-for-linkedhashmap-in-python

Ref: https://acadpubl.eu/hub/2018-119-12/articles/5/1211.pdf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(Data)
X_train_counts.shape

- Above ‘count_vect.fit_transform(twenty_train.data)’, create words/tokens dictionary. such that [n_samples, n_features].

#### Frequency–inverse document frequency(Tfidf):


It measures how important a word is to differentiate each category. It reduce the weightage of more common words like (stop words or common words) which occurs in all tweets. 

Code: https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-5-50b4e87d9bdd 

Ref: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.121.1424&rep=rep1&type=pdf 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

#### Generating Balanced Data using resample from Sklearn

In [ ]:
ASDA_data = []

for t, r in zip(Data,rating):
    ASDA_data.append([t,r])

ASDA_Crp = pd.DataFrame(ASDA_data, columns=['Tweets', 'rating'])

In [ ]:
print(ASDA_Crp.rating.value_counts())
# s2.value_counts().plot(kind='hist') 
ASDA_Crp.rating.value_counts().plot( kind="bar") 
plt.show()

** 1- Downsampling - Balanced Data

SRC: https://elitedatascience.com/imbalanced-classes

In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority_OBJ = ASDA_Crp[ASDA_Crp.rating=='OBJ']
df_majority_NEG = ASDA_Crp[ASDA_Crp.rating=='NEG']
df_majority_NEUTRAL = ASDA_Crp[ASDA_Crp.rating=='NEUTRAL']

df_minority_POS = ASDA_Crp[ASDA_Crp.rating=='POS']
 
# Downsample majority class
df_majority_OBJ_downsampled = resample(df_majority_OBJ, 
                                 replace=False,    # sample without replacement
                                 n_samples=799,     # to match minority class OBJ
                                 random_state=0) # reproducible results

df_majority_NEG_downsampled = resample(df_majority_NEG, 
                                 replace=False,    # sample without replacement 
                                 n_samples=799,     # to match minority class NEG
                                 random_state=0) # reproducible results

df_majority_NEUTRAL_downsampled = resample(df_majority_NEUTRAL, 
                                 replace=True,    # sample without replacement
                                 n_samples=799,     # to match minority class NATURAL
                                 random_state=0) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_OBJ_downsampled,df_majority_NEG_downsampled,df_majority_NEUTRAL_downsampled, df_minority_POS])
 
# Display new class counts
df_downsampled.rating.value_counts()

In [ ]:
df_downsampled.head()

In [ ]:
print(df_downsampled.rating.value_counts())
# s2.value_counts().plot(kind='hist') 
df_downsampled.rating.value_counts().plot( kind="bar") 
plt.show()

** 2- UpSampling - Balanced Data

In [ ]:
# Separate majority and minority classes
df_majority_OBJ = ASDA_Crp[ASDA_Crp.rating=='OBJ']

df_minority_NEG = ASDA_Crp[ASDA_Crp.rating=='NEG']
df_minority_NEUTRAL = ASDA_Crp[ASDA_Crp.rating=='NEUTRAL']
df_minority_POS = ASDA_Crp[ASDA_Crp.rating=='POS']
 

# Upsample minority class
df_minority_NEG_upsampled = resample(df_minority_NEG, 
                                 replace=True,     # sample with replacement
                                 n_samples=6691,    # to match majority class
                                 random_state=0) # reproducible results
# Upsample minority class
df_minority_NEUTRAL_upsampled = resample(df_minority_NEUTRAL, 
                                 replace=True,     # sample with replacement
                                 n_samples=6691,    # to match majority class
                                 random_state=0) # reproducible results
# Upsample minority class
df_minority_POS_upsampled = resample(df_minority_POS, 
                                 replace=True,     # sample with replacement
                                 n_samples=6691,    # to match majority class
                                 random_state=0) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority_OBJ, df_minority_POS_upsampled,df_minority_NEUTRAL_upsampled,df_minority_NEG_upsampled])
 
# Display new class counts
df_upsampled.rating.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64

In [ ]:
df_upsampled.head()

In [ ]:
print(df_upsampled.rating.value_counts())
# s2.value_counts().plot(kind='hist') 
df_upsampled.rating.value_counts().plot( kind="bar") 
plt.show()

In [ ]:
DataDistCollection = {'up_balanced': df_upsampled ,'down_balanced': df_downsampled , 'unBalanced': ASDA_Crp}

## 3. Prediction

### Step 4: Machine implementation using scikit-learn ###

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
models = [
    LogisticRegression(random_state=0),
    PassiveAggressiveClassifier(random_state=0),
    LinearSVC(tol=1e-3, random_state=0),
    Perceptron(tol=.2,random_state=0),
    BernoulliNB(binarize=0.5),
    SGDClassifier(loss="hinge", penalty="l2", random_state=0),
    KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    DecisionTreeClassifier(),
    MultinomialNB()
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, X_train_tfidf, rating, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
        
        
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df.head()

In [ ]:
cv_df[cv_df['accuracy'] > .68]

In [ ]:
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=10, jitter=True, edgecolor="gray", linewidth=2)

plt.show()

### Step 5: Fine tune the all the models ###  

In [ ]:
#this function to retrive sentimental count in each class
def GetCount(array):
    df = pd.DataFrame(array,columns= ["Class"])
    counts = df.Class.value_counts()
    OBJ = counts['OBJ']
    NEG = counts['NEG']
    NEUTRAL = counts['NEUTRAL']
    POS = counts['POS']
    return [OBJ, NEG, NEUTRAL ,POS]

In [ ]:
# Import train_test_split
from sklearn.cross_validation import train_test_split


gr = AraTweet()


classifiers_collection = {}
scores_collection = {}
dataframe_collection = {}
TweetCount_collection = {}

results = []

FeatuerDataSet = []
for key in DataDistCollection.keys():
    scores = list()
    # Split the 'features' and 'income' data into training and testing sets
    d_train, d_test, y_train, y_test = train_test_split(DataDistCollection[key].Tweets.values, 
                                                    DataDistCollection[key].rating.values, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

    # Show the results of the split
    # print("Training set has {} samples.".format(X_train.shape[0]))
    # print("Testing set has {} samples.".format(X_test.shape[0]))


    #---------------------------------------------------------------------------------
    TweetCount = [GetCount(y_train ), GetCount(y_test) ]
    df_tweetCount = pd.DataFrame(TweetCount,columns= ['OBJ', 'NEG', 'NEUTRAL' ,'POS']) 
    df_tweetCount['Dataset'] = ['Train Set','Test Set']
    df_tweetCount['name'] = key
    TweetCount_collection[key] = df_tweetCount
    #---------------------------------------------------------------------------------
    for feat_generator in Features_Generators:
            ####################################Features Generation#############################
    #         print("Features Generation:", feat_generator['name'])
            X_train = feat_generator['feat_generator'].fit_transform(d_train)
            X_test = feat_generator['feat_generator'].transform(d_test)


            for clf in classifiers:
                    print("\n------\n")
                    print("tuning: ", clf["name"] )


                    grid_fit = clf['tune_clf'].fit(X_train, y_train)
                    best_clf = grid_fit.best_estimator_
                    # Make predictions using the unoptimized and model
                    predictions = (clf["clf"].fit(X_train, y_train)).predict(X_test)
                    best_predictions = best_clf.predict(X_test)


                    # Report the before-and-afterscores
                    Clf_name = clf["name"] 

                    UnOpt_accuracy = accuracy_score(y_test, predictions)
                    UnOpt_fScore = f1_score(y_test, predictions,average='macro')
                    Opt_accuracy = accuracy_score(y_test, best_predictions)
                    Opt_fScore = f1_score(y_test, best_predictions,average='macro')


            #         clasifiers.append(Clf_name)

                    if (UnOpt_accuracy < Opt_accuracy ):
                        Best_Settings = grid_fit.best_params_
                        datarow = [Clf_name,key,feat_generator['name'],Best_Settings,UnOpt_accuracy , UnOpt_fScore , Opt_accuracy , Opt_fScore]
                        print(Clf_name,'  ',key , '  ',feat_generator['name'],'  ',Opt_accuracy,'  ' , Opt_fScore )
#                         (acc, tacc, support, f1 , df) = Evaluate_Result(best_predictions, y_test)
                    else:
                        Best_Settings = clf["clf"].get_params()
                        datarow = [Clf_name,key,feat_generator['name'],Best_Settings,Opt_accuracy , Opt_fScore , UnOpt_accuracy , UnOpt_fScore]
#                         (acc, tacc, support, f1 , df) = Evaluate_Result(predictions, y_test)
                        
                        print(Clf_name,'  ',key , '  ',feat_generator['name'],'  ',UnOpt_accuracy,'  ' , UnOpt_fScore )
                        
                    
                    results.append(datarow)


                    score = dict(data=key,
                                             feat_generator=feat_generator['name'],
                                             clf=clf['name'],
                                             # feat_ext=feat_ext['name'],
                                             f1=f1,
                                             acc=acc,
                                             tacc=tacc)

                    df['Classifier'] =  clf["name"]
                    df['feauter_generator'] = feat_generator['name']

                    df['acc'] = acc
                    df['f1']=f1

                    dataframe_collection[ key + '-' + clf["name"] + '-' + feat_generator['name'] ]  = df
    #                         print(clf["name"],df['feauter_generator'] )
                    scores.append(score)
        
    for key in dataframe_collection.keys():
        print("\n" +"="*40)
        print(key)
        print("-"*40)
        print(dataframe_collection[key])

        df_Classifiers = pd.DataFrame(classifiers)
        classifiers_collection[key] = df_Classifiers

    #     print("-"*40)
    #     print(df_Classifiers.head())

        df_s = pd.DataFrame(scores)
        scores_collection[key] = df_s
        
df_feauterCount = pd.DataFrame(FeatuerDataSet, columns=['DataSet_Name','Feauter_Generator','Feauters_Count'])

## 4. Evaluation

### Step 6: Performance Measures for Classifieres ###  

In [ ]:
df_resutlts = pd.DataFrame(results,columns=['clasifiers','Data','Feauter_Generator' ,'Best_Settings' , 'UnOpt_accuracys' , 'UnOpt_fScores' , 'Opt_accuracys' , 'Opt_fScores'])
# df_resutlts.index = df_resutlts.clasifiers.get_values()
df_resutlts.head()

In [ ]:
df_resutlts[df_resutlts['Opt_accuracys'] > .97]

In [ ]:
subset = df_resutlts[df_resutlts['Opt_accuracys'] > .70]

d = subset[subset['Opt_fScores'] < .50]

d

In [ ]:

dataset = df_resutlts[['Opt_accuracys','UnOpt_accuracys']]
ax = dataset.plot(figsize=(20,10))
ax.set_xticklabels( df_resutlts.clasifiers.get_values())
# ax.set_xticks([1,2,3,4,5,6,7,8,9])

# ax.set_xlim([0, 10])
# ax.set_xlabel("Classifiers" )
# ax.set_ylabel("Accuracy")
# ax.set_title("Visualizing Results using Accuracy")